In [ ]:
import boto3

sns = boto3.client('sns')
sqs = boto3.client('sqs')

In [ ]:
import os
account_no = os.environ['ACCOUNT_NO']

In [ ]:
ct = sns.create_topic(
    Name='topicOne'
)
print(ct)
topic_arn1 = ct['TopicArn']

In [ ]:
ct = sns.create_topic(
    Name='topicTwo'
)
print(ct)
topic_arn2 = ct['TopicArn']

In [ ]:
cq = sqs.create_queue(
    QueueName='sampleQueue'
)
print(cq)
queue_url = cq['QueueUrl']
queue_arn = 'arn:aws:sqs:us-east-1:' + account_no + ':sampleQueue'

In [ ]:
sns.subscribe(
    TopicArn=topic_arn1,
    Protocol='sqs',
    Endpoint=queue_arn
)

In [ ]:
sns.subscribe(
    TopicArn=topic_arn2,
    Protocol='sqs',
    Endpoint=queue_arn
)

In [ ]:
def allow_sns_to_write_to_sqs(topicarn, queuearn):
    policy_document = """{{
  "Version":"2012-10-17",
  "Statement":[
    {{
      "Sid":"MyPolicy",
      "Effect":"Allow",
      "Principal" : {{"AWS" : "*"}},
      "Action":"SQS:SendMessage",
      "Resource": "{}",
      "Condition":{{
        "ArnLike":{{
          "aws:SourceArn": "{}"
        }}
      }}
    }}
  ]
}}""".format(queuearn, topicarn)

    return policy_document

In [ ]:
#policy_json = allow_sns_to_write_to_sqs(topic_arn1, queue_arn)
policy_json = allow_sns_to_write_to_sqs('arn:aws:sns:us-east-1:' + account_no + ':*', queue_arn)

response = sqs.set_queue_attributes(
    QueueUrl = queue_url,
    Attributes = {
        'Policy' : policy_json
    }
)
print(response)

In [ ]:
sns.publish(
    TopicArn='arn:aws:sns:us-east-1:' + account_no + ':topicOne',
    Message='TOPIC ONE ONE ONE ONE ONE ONE ONE'
)

In [ ]:
sns.publish(
    TopicArn='arn:aws:sns:us-east-1:' + account_no + ':topicTwo',
    Message='TOPIC TWO TOPIC TWO'
)

In [ ]:
qurl = 'https://queue.amazonaws.com/' + account_no + '/sampleQueue'
rr = sqs.receive_message(
    QueueUrl=qurl
)
print(rr)

In [ ]:
msg = rr['Messages'][0]
rhandle = msg['ReceiptHandle']

dm = sqs.delete_message(
    QueueUrl=qurl,
    ReceiptHandle=rhandle
)

print(dm)

In [ ]:
sqs.delete_queue(
    QueueUrl='https://queue.amazonaws.com/' + account_no + '/sampleQueue'
)

In [ ]:
sns.delete_topic(
    TopicArn='arn:aws:sns:us-east-1:' + account_no + ':topicOne'
)